In [ ]:
import pandas as pd
import numpy as np
import keras

from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, Conv1D, MaxPooling1D, Flatten
from keras.metrics import CategoricalAccuracy, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.utils.np_utils import to_categorical, normalize
from sklearn.utils import shuffle
import keras
from keras.models import Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Dense, Dropout
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, f_classif, SelectFpr
import operator
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
import math

print('Imports complete')

Imports complete


In [ ]:
path = '/content/drive/MyDrive/cleaneddata.csv'
df = pd.read_csv(path)

In [ ]:
df.head()

,Flow ID,Source Port,Destination Port,Timestamp,Flow Duration,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd Packets/s,Max Packet Length,Packet Length Mean,Packet Length Std,URG Flag Count,CWE Flag Count,ECE Flag Count,Avg Fwd Segment Size,Fwd Header Length.1,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,172.217.2.174-10.42.0.211-443-51023-6,51023,443,16/06/2017 03:55:47,151054,1076,4575,179.333333,321.621931,571.875000,679.532284,37410.462480,92.682087,11619.53846,14541.15588,49105,24,101887,20377.4,12821.55074,30425,111,128516,18359.42857,24038.55786,54822,24,200.0,52.961193,0,1418,1,0,0,1,179.333333,6,1076,8,4575,353,3,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,172.217.2.174-10.42.0.211-443-51023-6,51023,443,16/06/2017 03:55:47,349,23,0,11.500000,16.263456,0.000000,0.000000,65902.578800,5730.659026,349.00000,0.00000,349,349,349,349.0,0.00000,349,349,0,0.00000,0.00000,0,0,64.0,0.000000,0,23,0,1,0,0,11.500000,2,23,0,0,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,172.217.12.174-10.42.0.211-443-34259-6,34259,443,16/06/2017 03:55:52,119,23,0,11.500000,16.263456,0.000000,0.000000,193277.310900,16806.722690,119.00000,0.00000,119,119,119,119.0,0.00000,119,119,0,0.00000,0.00000,0,0,64.0,0.000000,0,23,0,1,0,0,11.500000,2,23,0,0,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,172.217.10.74-10.42.0.211-443-55509-6,55509,443,16/06/2017 03:55:53,37055,31,0,31.000000,0.000000,0.000000,0.000000,836.594252,53.973823,37055.00000,0.00000,37055,37055,0,0.0,0.00000,0,0,0,0.00000,0.00000,0,0,32.0,26.986911,0,31,0,1,0,1,31.000000,1,31,1,0,391,0,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,172.217.2.174-10.42.0.211-443-44852-6,44852,443,16/06/2017 03:55:58,178727,1313,307,218.833333,331.306152,43.857143,75.366722,9064.103353,72.736632,14893.91667,18532.64075,62243,55,125550,25110.0,28468.23202,70464,172,156011,26001.83333,24955.90182,67743,55,200.0,39.165879,0,753,1,0,0,1,218.833333,6,1313,7,307,352,3,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [ ]:
df.tail()

,Flow ID,Source Port,Destination Port,Timestamp,Flow Duration,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd Packets/s,Max Packet Length,Packet Length Mean,Packet Length Std,URG Flag Count,CWE Flag Count,ECE Flag Count,Avg Fwd Segment Size,Fwd Header Length.1,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
1048570,10.42.0.1-10.42.0.42-53-52113-17,52113,53,28/08/2017 03:29:57,377347,44,194,44.0,0.000000,194.0,0.0,630.719205,5.300161,377347.00,0.00,377347,377347,0,0.00,0.00,0,0,0,0.0,0.0,0,0,40.0,2.650081,44,194,0,0,0,1,44.0,1,44,1,194,-1,0,40,0.0,0.0,0,0,0.0,0.0,0,0,RANSOMWARE_SVPENG
1048571,180.149.153.68-10.42.0.42-80-57160-6,80,57160,28/08/2017 04:10:18,2261,0,0,0.0,0.000000,0.0,0.0,0.000000,884.564352,2261.00,0.00,2261,2261,0,0.00,0.00,0,0,0,0.0,0.0,0,0,32.0,442.282176,0,0,0,1,1,1,0.0,1,0,1,0,1369,0,32,0.0,0.0,0,0,0.0,0.0,0,0,RANSOMWARE_SVPENG
1048572,10.42.0.1-10.42.0.42-53-17292-17,17292,53,28/08/2017 04:13:40,840872,32,207,32.0,0.000000,207.0,0.0,284.228753,2.378483,840872.00,0.00,840872,840872,0,0.00,0.00,0,0,0,0.0,0.0,0,0,40.0,1.189242,32,207,0,0,0,1,32.0,1,32,1,207,-1,0,40,0.0,0.0,0,0,0.0,0.0,0,0,RANSOMWARE_SVPENG
1048573,10.42.0.42-120.132.42.172-44600-80-6,80,44600,28/08/2017 04:08:08,2394,0,0,0.0,0.000000,0.0,0.0,0.000000,835.421888,2394.00,0.00,2394,2394,0,0.00,0.00,0,0,0,0.0,0.0,0,0,32.0,417.710944,0,0,0,1,1,1,0.0,1,0,1,0,1369,0,32,0.0,0.0,0,0,0.0,0.0,0,0,RANSOMWARE_SVPENG
1048574,204.13.194.242-10.42.0.42-443-42974-6,42974,443,28/08/2017 03:34:09,114924911,62,0,15.5,17.897858,0.0,0.0,0.539483,0.034805,38308303.67,66316512.47,114884013,938,114924911,38308303.67,66316512.47,114884013,938,0,0.0,0.0,0,0,80.0,0.000000,0,31,0,1,1,0,15.5,4,62,0,0,-1,1,20,39960.0,0.0,39960,39960,114884013.0,0.0,114884013,114884013,RANSOMWARE_SVPENG


In [ ]:
df['Label'].value_counts()
dep_var = 'Label'
df[dep_var].value_counts()

BENIGN                   410548
RANSOMWARE               348943
RANSOMWARE_PORNDROID      46082
RANSOMWARE_KOLER          44555
RANSOMWARE_RANSOMBO       39859
RANSOMWARE_CHARGER        39551
RANSOMWARE_SIMPLOCKER     36340
RANSOMWARE_SVPENG         27003
RANSOMWARE_JISUT          25672
RANSOMWARE_LOCKERPIN      25307
RANSOMWARE_PLETOR          4715
Name: Label, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 52 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   Flow ID                      1048575 non-null  object 
 1   Source Port                  1048575 non-null  int64  
 2   Destination Port             1048575 non-null  int64  
 3   Timestamp                    1048575 non-null  object 
 4   Flow Duration                1048575 non-null  int64  
 5   Total Length of Fwd Packets  1048575 non-null  int64  
 6   Total Length of Bwd Packets  1048575 non-null  int64  
 7   Fwd Packet Length Mean       1048575 non-null  float64
 8   Fwd Packet Length Std        1048575 non-null  float64
 9   Bwd Packet Length Max        1048575 non-null  float64
 10  Bwd Packet Length Std        1048575 non-null  float64
 11  Flow Bytes/s                 1048575 non-null  float64
 12  Flow Packets/s               1048575 non-n

In [ ]:
df.columns

Index(['Flow ID', 'Source Port', 'Destination Port', 'Timestamp',
       'Flow Duration', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd Packets/s', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'URG Flag Count', 'CWE Flag Count',
       'ECE Flag Count', 'Avg Fwd Segment Size', 'Fwd Header Length.1',
       'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
       'Init_Win_bytes_backward', 'min_seg_size_forward', 'Active Mean',
       'Active Std', 'Active Max', 'Active Min', 'Idle M

In [ ]:
y = df[dep_var]
del df[dep_var]

In [ ]:

X = (df - df.mean()) / (df.max() - df.min())
X = X.values
#X = df
print(X.shape)

(1048575, 51)


In [ ]:
valid_cols = ['Flow ID', 'Source IP', 'Source Port',
       'Destination IP', 'Destination Port', 'Protocol', 'Timestamp',
       'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
       'Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk',
       'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk',
       'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
       'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
       'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
       'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
       'Idle Std', 'Idle Max', 'Idle Min', 'Label']
for col in X.columns:
    #print(X[col].dtype)
    if X[col].dtype == 'object' or col not in valid_cols:
        del X[col]
        #print('dropped')

In [ ]:
print(X.shape)
X1=X.copy()
X2=X.copy()
y1=y.copy()
y2=y.copy()

(1048575, 49)


In [ ]:
df1=df.copy()
df2=df.copy()

In [ ]:
# Technique from https://machinelearningmastery.com/feature-selection-machine-learning-python/
# feature extraction
test = SelectFpr(score_func=f_classif, alpha=0.05)
fit = test.fit(X, y)
# summarize scores
np.set_printoptions(precision=3)
#print(fit.scores_)
dictt = []
i=0
for col in X.columns:
    score = fit.scores_[i]
    #print(score)
    i+=1
    if not math.isnan(float(score)):
        dictt.append([col, score])    
    
#dictt = sorted(dictt, key=operator.itemgetter(1), reverse=True)
dictt.sort(key=lambda arr : float(arr[1]), reverse=True)

outstr = ''

i=1
for col, score in dictt[:20]:
    #if not math.isnan(float(score)):
    print('{}\t{}\t{}'.format(i, col, score))
    i+=1
    
    outstr += '\'' + col + '\', '

print(outstr)


1	Active Max	677.6000614382294
2	Destination Port	617.1654480819002
3	Active Std	531.3617253049655
4	Active Mean	487.7271720509815
5	Init_Win_bytes_backward	466.4327605491759
6	URG Flag Count	446.0371823679936
7	Total Length of Fwd Packets	406.4139291026289
8	Fwd Header Length.1	406.4139291026289
9	Source Port	397.7716672219904
10	Flow IAT Mean	370.94468404250125
11	Flow IAT Min	289.4728599246458
12	Fwd IAT Total	287.69516928161494
13	Active Min	275.8639691756266
14	Fwd IAT Max	244.84775107860798
15	Fwd IAT Mean	243.04861113906384
16	Packet Length Std	242.16766421699666
17	Fwd IAT Min	225.10398521933095
18	Fwd IAT Std	212.62156667885895
19	Avg Fwd Segment Size	210.57948357778236
20	Flow IAT Max	207.23978764730646
'Active Max', 'Destination Port', 'Active Std', 'Active Mean', 'Init_Win_bytes_backward', 'URG Flag Count', 'Total Length of Fwd Packets', 'Fwd Header Length.1', 'Source Port', 'Flow IAT Mean', 'Flow IAT Min', 'Fwd IAT Total', 'Active Min', 'Fwd IAT Max', 'Fwd IAT Mean', 'Pack

In [ ]:
cols_to_keep = ['Active Max', 'Destination Port', 'Active Std', 'Active Mean', 'Init_Win_bytes_backward', 'URG Flag Count', 'Total Length of Fwd Packets', 'Fwd Header Length.1', 'Source Port', 'Flow IAT Mean', 'Flow IAT Min', 'Fwd IAT Total', 'Active Min', 'Fwd IAT Max', 'Fwd IAT Mean', 'Packet Length Std', 'Fwd IAT Min', 'Fwd IAT Std', 'Avg Fwd Segment Size', 'Flow IAT Max',
                    'Label']
print('Dropping unnecessary columns...')
df.drop(columns=[col for col in df.columns if col not in cols_to_keep], inplace=True, errors='raise') 
df.dropna(inplace=True)


Dropping unnecessary columns...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 20 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   Source Port                  1048575 non-null  int64  
 1   Destination Port             1048575 non-null  int64  
 2   Total Length of Fwd Packets  1048575 non-null  int64  
 3   Flow IAT Mean                1048575 non-null  float64
 4   Flow IAT Max                 1048575 non-null  int64  
 5   Flow IAT Min                 1048575 non-null  int64  
 6   Fwd IAT Total                1048575 non-null  int64  
 7   Fwd IAT Mean                 1048575 non-null  float64
 8   Fwd IAT Std                  1048575 non-null  float64
 9   Fwd IAT Max                  1048575 non-null  int64  
 10  Fwd IAT Min                  1048575 non-null  int64  
 11  Packet Length Std            1048575 non-null  int64  
 12  URG Flag Count               1048575 non-n

In [ ]:
print(X.shape)
print(y.shape)

(1048575, 20)
(1048575,)


In [ ]:
X = normalize( ( df.loc[:, df.columns != dep_var] ).values )

# One-Hot Encoding for the target classification
label_encoder = LabelEncoder()
#y = label_encoder.fit_transform(y)
y = y.map(lambda classif : 0 if classif == 'BENIGN' else 1)
y = keras.utils.to_categorical(y, num_classes=y.nunique())

print('Data:')
print(X[:5, :])


Data:
[[3.909e-01 3.394e-03 8.243e-03 8.902e-02 3.762e-01 1.839e-04 7.806e-01
  1.561e-01 9.823e-02 2.331e-01 8.504e-04 0.000e+00 0.000e+00 4.597e-05
  8.243e-03 2.298e-05 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [9.998e-01 8.681e-03 4.507e-04 6.839e-03 6.839e-03 6.839e-03 6.839e-03
  6.839e-03 0.000e+00 6.839e-03 6.839e-03 1.960e-05 0.000e+00 3.919e-05
  4.507e-04 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [9.999e-01 1.293e-02 6.713e-04 3.473e-03 3.473e-03 3.473e-03 3.473e-03
  3.473e-03 0.000e+00 3.473e-03 3.473e-03 2.919e-05 0.000e+00 5.837e-05
  6.713e-04 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [6.541e-01 5.221e-03 3.653e-04 4.367e-01 4.367e-01 4.367e-01 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.178e-05 0.000e+00 1.178e-05
  3.653e-04 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [2.667e-01 2.634e-03 7.808e-03 8.857e-02 3.701e-01 3.271e-04 7.466e-01
  1.493e-01 1.693e-01 4.190e-01 1.023e-03 0.000e+00 0.000e+00 3.568e-05
  7.808e-03 1.784e-05 

In [ ]:
X.shape

(1048575, 20)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True, stratify=y, random_state=1)

In [ ]:
total_folds=10
fold_num=1

In [ ]:


accuracy = CategoricalAccuracy() 	# Will change this to Categorical if the target classification is categorical
tp = TruePositives()		# These could be collected with a confusion matrix, however translating back
tn = TrueNegatives()		#  and forth from an image may be frustrating (it was last time I did it)
fp = FalsePositives()
fn = FalseNegatives()
metrics = [accuracy, tp, tn, fp, fn]

# The model must be reinitialized otherwise the model will have trained on all of the data (that wouldn't be true 10-fold cv)
model = Sequential()
model.add(Dense(64, input_shape=(20 ,))) 	# Input layer, needs same shape as input data (9 values 1D)
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(128))			# Hidden layer of nodes
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(32))			# Hidden layer of nodes
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(8))			# Hidden layer of nodes
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(2, activation='softmax'))			# Output layer of 3 nodes 

# "Configures the model for training"
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)
    
# Fit and test the model 
model.fit(x=X_train, y=y_train, epochs=20, batch_size=512, verbose=0, validation_data=(X_test, y_test))
    
# Evaluate the performance of the model on the test set
scores = model.evaluate(X_test, y_test, verbose=2)
acc, loss, tpn, tnn, fpn, fnn = scores[1]*100, scores[0]*100, scores[2], scores[3], scores[4], scores[5]
totaln = tpn + tnn + fpn + fnn
print('Baseline: accuracy: {:.2f}%'.format(acc))
print('\tTrue Positive Rate: {}'.format(tpn/totaln))
print('\tTrue Negative Rate: {}'.format(tnn/totaln))
print('\tFalse Positive Rate: {}'.format(fpn/totaln))
print('\tFalse Negative Rate: {}'.format(fnn/totaln))

Baseline: accuracy: 89.41%
	True Positive Rate: 0.45223593330058086
	True Negative Rate: 0.44223593330058077
	False Positive Rate: 0.06776406669941916
	False Negative Rate: 0.05845475825795451


In [ ]:
  # Technique from https://machinelearningmastery.com/feature-selection-machine-learning-python/
# feature extraction
test = SelectFpr(score_func=f_classif, alpha=0.05)
fit = test.fit(X1, y1)
# summarize scores
np.set_printoptions(precision=3)
#print(fit.scores_)
dictt = []
i=0
for col in X1.columns:
    score = fit.scores_[i]
    #print(score)
    i+=1
    if not math.isnan(float(score)):
        dictt.append([col, score])    
    
#dictt = sorted(dictt, key=operator.itemgetter(1), reverse=True)
dictt.sort(key=lambda arr : float(arr[1]), reverse=True)

outstr = ''

i=1
for col, score in dictt[:15]:
    #if not math.isnan(float(score)):
    print('{}\t{}\t{}'.format(i, col, score))
    i+=1
    
    outstr += '\'' + col + '\', '

print(outstr)


1	Active Max	677.6000614382294
2	Destination Port	617.1654480819002
3	Active Std	531.3617253049655
4	Active Mean	487.7271720509815
5	Init_Win_bytes_backward	466.4327605491759
6	URG Flag Count	446.0371823679936
7	Total Length of Fwd Packets	406.4139291026289
8	Fwd Header Length.1	406.4139291026289
9	Source Port	397.7716672219904
10	Flow IAT Mean	370.94468404250125
11	Flow IAT Min	289.4728599246458
12	Fwd IAT Total	287.69516928161494
13	Active Min	275.8639691756266
14	Fwd IAT Max	244.84775107860798
15	Fwd IAT Mean	243.04861113906384
'Active Max', 'Destination Port', 'Active Std', 'Active Mean', 'Init_Win_bytes_backward', 'URG Flag Count', 'Total Length of Fwd Packets', 'Fwd Header Length.1', 'Source Port', 'Flow IAT Mean', 'Flow IAT Min', 'Fwd IAT Total', 'Active Min', 'Fwd IAT Max', 'Fwd IAT Mean', 


In [ ]:
cols_to_keep = ['Active Max', 'Destination Port', 'Active Std', 'Active Mean', 'Init_Win_bytes_backward', 'URG Flag Count', 'Total Length of Fwd Packets', 'Fwd Header Length.1', 'Source Port', 'Flow IAT Mean', 'Flow IAT Min', 'Fwd IAT Total', 'Active Min', 'Fwd IAT Max', 'Fwd IAT Mean',  
                    'Label']
print('Dropping unnecessary columns...')
df1.drop(columns=[col for col in df1.columns if col not in cols_to_keep], inplace=True, errors='raise') 
df1.dropna(inplace=True)

Dropping unnecessary columns...


In [ ]:
X1 = normalize( ( df1.loc[:, df1.columns != dep_var] ).values )

# One-Hot Encoding for the target classification
label_encoder = LabelEncoder()
#y = label_encoder.fit_transform(y)
y1 = y1.map(lambda classif : 0 if classif == 'BENIGN' else 1)
y1 = keras.utils.to_categorical(y1, num_classes=y1.nunique())

print('Data:')
print(X1[:5, :])


Data:
[[4.243e-01 3.684e-03 8.947e-03 9.662e-02 1.996e-04 8.472e-01 1.694e-01
  2.530e-01 0.000e+00 8.947e-03 2.495e-05 0.000e+00 0.000e+00 0.000e+00
  0.000e+00]
 [9.998e-01 8.681e-03 4.507e-04 6.839e-03 6.839e-03 6.839e-03 6.839e-03
  6.839e-03 0.000e+00 4.507e-04 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00]
 [9.999e-01 1.293e-02 6.713e-04 3.473e-03 3.473e-03 3.473e-03 3.473e-03
  3.473e-03 0.000e+00 6.713e-04 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00]
 [7.271e-01 5.803e-03 4.061e-04 4.854e-01 4.854e-01 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 4.061e-04 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00]
 [2.920e-01 2.884e-03 8.548e-03 9.696e-02 3.581e-04 8.173e-01 1.635e-01
  4.587e-01 0.000e+00 8.548e-03 1.953e-05 0.000e+00 0.000e+00 0.000e+00
  0.000e+00]]


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 15 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   Source Port                  1048575 non-null  int64  
 1   Destination Port             1048575 non-null  int64  
 2   Total Length of Fwd Packets  1048575 non-null  int64  
 3   Flow IAT Mean                1048575 non-null  float64
 4   Flow IAT Min                 1048575 non-null  int64  
 5   Fwd IAT Total                1048575 non-null  int64  
 6   Fwd IAT Mean                 1048575 non-null  float64
 7   Fwd IAT Max                  1048575 non-null  int64  
 8   URG Flag Count               1048575 non-null  int64  
 9   Fwd Header Length.1          1048575 non-null  int64  
 10  Init_Win_bytes_backward      1048575 non-null  int64  
 11  Active Mean                  1048575 non-null  float64
 12  Active Std                   1048575 non-n

In [ ]:
print(X1.shape)
print(y1.shape)

(1048575, 15)
(1048575, 2)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.33, shuffle=True, stratify=y1, random_state=1)

In [ ]:
# for train_idx, test_idx in StratifiedKFold(n_splits=total_folds, shuffle=True, random_state=1).split(data_x, data_y):
#     if fold_num == 1:
#         data_y = to_categorical(data_y)
    
#     print('Fold {}/{}'.format(fold_num, total_folds))
#     fold_num += 1

#     # Set up the training and testing sets
#     X_train, X_test = data_x[train_idx], data_x[test_idx]
#     y_train, y_test = data_y[train_idx], data_y[test_idx]


accuracy = CategoricalAccuracy() 	# Will change this to Categorical if the target classification is categorical
tp = TruePositives()		# These could be collected with a confusion matrix, however translating back
tn = TrueNegatives()		#  and forth from an image may be frustrating (it was last time I did it)
fp = FalsePositives()
fn = FalseNegatives()
metrics = [accuracy, tp, tn, fp, fn]

# The model must be reinitialized otherwise the model will have trained on all of the data (that wouldn't be true 10-fold cv)
model = Sequential()
model.add(Dense(64, input_shape=(15 ,))) 	# Input layer, needs same shape as input data (9 values 1D)
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(128))			# Hidden layer of nodes
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(32))			# Hidden layer of nodes
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(8))			# Hidden layer of nodes
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(2, activation='softmax'))			# Output layer of 3 nodes 

# "Configures the model for training"
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)
    
# Fit and test the model 
model.fit(x=X_train, y=y_train, epochs=20, batch_size=512, verbose=0, validation_data=(X_test, y_test))
    
# Evaluate the performance of the model on the test set
scores = model.evaluate(X_test, y_test, verbose=2)
acc, loss, tpn, tnn, fpn, fnn = scores[1]*100, scores[0]*100, scores[2], scores[3], scores[4], scores[5]
totaln = tpn + tnn + fpn + fnn
print('Baseline: accuracy: {:.2f}%'.format(acc))
print('\tTrue Positive Rate: {}'.format(tpn/totaln))
print('\tTrue Negative Rate: {}'.format(tnn/totaln))
print('\tFalse Positive Rate: {}'.format(fpn/totaln))
print('\tFalse Negative Rate: {}'.format(fnn/totaln))




Baseline: accuracy: 88.31%
	True Positive Rate: 0.4415452417420455
	True Negative Rate: 0.44115452417420455
	False Positive Rate: 0.06845475825795452
	False Negative Rate: 0.06845475825795458


In [ ]:
# Technique from https://machinelearningmastery.com/feature-selection-machine-learning-python/
# feature extraction
test = SelectFpr(score_func=f_classif, alpha=0.05)
fit = test.fit(X2, y2)
# summarize scores
np.set_printoptions(precision=3)
#print(fit.scores_)
dictt = []
i=0
for col in X2.columns:
    score = fit.scores_[i]
    #print(score)
    i+=1
    if not math.isnan(float(score)):
        dictt.append([col, score])    
    
#dictt = sorted(dictt, key=operator.itemgetter(1), reverse=True)
dictt.sort(key=lambda arr : float(arr[1]), reverse=True)

outstr = ''

i=1
for col, score in dictt[:10]:
    #if not math.isnan(float(score)):
    print('{}\t{}\t{}'.format(i, col, score))
    i+=1
    
    outstr += '\'' + col + '\', '

print(outstr)


1	Active Max	677.6000614382294
2	Destination Port	617.1654480819002
3	Active Std	531.3617253049655
4	Active Mean	487.7271720509815
5	Init_Win_bytes_backward	466.4327605491759
6	URG Flag Count	446.0371823679936
7	Total Length of Fwd Packets	406.4139291026289
8	Fwd Header Length.1	406.4139291026289
9	Source Port	397.7716672219904
10	Flow IAT Mean	370.94468404250125
'Active Max', 'Destination Port', 'Active Std', 'Active Mean', 'Init_Win_bytes_backward', 'URG Flag Count', 'Total Length of Fwd Packets', 'Fwd Header Length.1', 'Source Port', 'Flow IAT Mean', 


In [ ]:
cols_to_keep = ['Active Max', 'Destination Port', 'Active Std', 'Active Mean', 'Init_Win_bytes_backward', 'URG Flag Count', 'Total Length of Fwd Packets', 'Fwd Header Length.1', 'Source Port', 'Flow IAT Mean', 
                    'Label']
print('Dropping unnecessary columns...')
df2.drop(columns=[col for col in df2.columns if col not in cols_to_keep], inplace=True, errors='raise') 
df2.dropna(inplace=True)

Dropping unnecessary columns...


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 10 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   Source Port                  1048575 non-null  int64  
 1   Destination Port             1048575 non-null  int64  
 2   Total Length of Fwd Packets  1048575 non-null  int64  
 3   Flow IAT Mean                1048575 non-null  float64
 4   URG Flag Count               1048575 non-null  int64  
 5   Fwd Header Length.1          1048575 non-null  int64  
 6   Init_Win_bytes_backward      1048575 non-null  int64  
 7   Active Mean                  1048575 non-null  float64
 8   Active Std                   1048575 non-null  float64
 9   Active Max                   1048575 non-null  int64  
dtypes: float64(3), int64(7)
memory usage: 88.0 MB


In [ ]:
print(X2.shape)

(1048575, 49)


In [ ]:
X2 = normalize( ( df2.loc[:, df2.columns != dep_var] ).values )

# One-Hot Encoding for the target classification
label_encoder = LabelEncoder()
#y = label_encoder.fit_transform(y)
y2 = y2.map(lambda classif : 0 if classif == 'BENIGN' else 1)
y2 = keras.utils.to_categorical(y2, num_classes=y2.nunique())

print('Data:')
print(X2[:5, :])

Data:
[[9.746e-01 8.462e-03 2.055e-02 2.219e-01 0.000e+00 2.055e-02 5.730e-05
  0.000e+00 0.000e+00 0.000e+00]
 [9.999e-01 8.682e-03 4.507e-04 6.840e-03 0.000e+00 4.507e-04 0.000e+00
  0.000e+00 0.000e+00 0.000e+00]
 [9.999e-01 1.293e-02 6.713e-04 3.473e-03 0.000e+00 6.713e-04 0.000e+00
  0.000e+00 0.000e+00 0.000e+00]
 [8.317e-01 6.637e-03 4.645e-04 5.552e-01 0.000e+00 4.645e-04 0.000e+00
  0.000e+00 0.000e+00 0.000e+00]
 [9.483e-01 9.366e-03 2.776e-02 3.149e-01 0.000e+00 2.776e-02 6.343e-05
  0.000e+00 0.000e+00 0.000e+00]]


In [ ]:
X2.shape

(1048575, 10)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.33, shuffle=True, stratify=y2, random_state=1)

In [ ]:
# for train_idx, test_idx in StratifiedKFold(n_splits=total_folds, shuffle=True, random_state=1).split(data_x, data_y):
#     if fold_num == 1:
#         data_y = to_categorical(data_y)
    
#     print('Fold {}/{}'.format(fold_num, total_folds))
#     fold_num += 1

#     # Set up the training and testing sets
#     X_train, X_test = data_x[train_idx], data_x[test_idx]
#     y_train, y_test = data_y[train_idx], data_y[test_idx]


accuracy = CategoricalAccuracy() 	# Will change this to Categorical if the target classification is categorical
tp = TruePositives()		# These could be collected with a confusion matrix, however translating back
tn = TrueNegatives()		#  and forth from an image may be frustrating (it was last time I did it)
fp = FalsePositives()
fn = FalseNegatives()
metrics = [accuracy, tp, tn, fp, fn]

# The model must be reinitialized otherwise the model will have trained on all of the data (that wouldn't be true 10-fold cv)
model = Sequential()
model.add(Dense(64, input_shape=(10 ,))) 	# Input layer, needs same shape as input data (9 values 1D)
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(128))			# Hidden layer of nodes
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(32))			# Hidden layer of nodes
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(8))			# Hidden layer of nodes
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(2, activation='softmax'))			# Output layer of 3 nodes 

# "Configures the model for training"
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)
    
# Fit and test the model 
model.fit(x=X_train, y=y_train, epochs=20, batch_size=512, verbose=0, validation_data=(X_test, y_test))
    
# Evaluate the performance of the model on the test set
scores = model.evaluate(X_test, y_test, verbose=2)
acc, loss, tpn, tnn, fpn, fnn = scores[1]*100, scores[0]*100, scores[2], scores[3], scores[4], scores[5]
totaln = tpn + tnn + fpn + fnn
print('Baseline: accuracy: {:.2f}%'.format(acc))
print('\tTrue Positive Rate: {}'.format(tpn/totaln))
print('\tTrue Negative Rate: {}'.format(tnn/totaln))
print('\tFalse Positive Rate: {}'.format(fpn/totaln))
print('\tFalse Negative Rate: {}'.format(fnn/totaln))



Baseline: accuracy: 88.22%
	True Positive Rate: 0.44609340230615845
	True Negative Rate: 0.44609340230615845
	False Positive Rate: 0.06390659769384158
	False Negative Rate: 0.06390659769384158
